In [77]:
import pandas as pd
import math
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix  
from sklearn.svm import SVC
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import warnings 
from datetime import date

from scipy import stats

from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns 


from sklearn.feature_selection import f_classif # anova
from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import SelectPercentile

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor 
# инструменты для оценки точности модели
from sklearn import metrics 

In [78]:
df = pd.read_csv('./clean_data.csv')
df = df.loc[(df['token_age_b'] <= 60) & (df['token_age_b'] >= 6)]


In [79]:
df.columns

Index(['price_b', 'token_age_b', 'token_age_s', 'buys_m5', 'sells_m5',
       'buys_h1', 'sells_h1', 'volume_m5', 'volume_h1', 'price_change_m5',
       'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
       'market_cap', 'is_telegram', 'is_twitter', 'is_website', 'boosts',
       'telegram_members', 'twitter_days', 'twitter_tweets',
       'twitter_followers', 'twitter_smart_followers', 'closing_date',
       'sns_held_all', 'sns_sold_some', 'sns_sold_all', 'PNL', 'PNL_10',
       'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50', 'PNL_loss_5', 'PNL_loss_10',
       'price_change_check', 'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought',
       'sns_f10_sold', 'sns_no_bought', 'sns_no_sold', 'tt_no_bought',
       'tt_no_sold', 'sns_pnl_loss', 'tt_pnl_loss'],
      dtype='object')

In [80]:
y = df["PNL_50"].copy()
x = df.drop(columns=[
    # 'Unnamed: 53',
    #    'Unnamed: 54',
    
    #"transfers", 'is_mutable_metadata',
    'price_change_h6', 'price_change_h24', 'price_change_check', 'closing_date',
    #'volume_h1',
    #'price_change_h1',
    #'buys_h1', 'sells_h1',
    'PNL', 'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50', 'token_age_s',
    'telegram_members', 'twitter_days', 
    'twitter_tweets', 'twitter_followers', 'twitter_smart_followers', 
    # 'sns_most_common', 'sns_most_common_except_0',
    #    'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought', 'sns_no_bought',
    #    'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
    #    'tt_pnl_loss'
    ])

In [81]:
x.columns

Index(['price_b', 'token_age_b', 'buys_m5', 'sells_m5', 'buys_h1', 'sells_h1',
       'volume_m5', 'volume_h1', 'price_change_m5', 'price_change_h1',
       'liquidity', 'market_cap', 'is_telegram', 'is_twitter', 'is_website',
       'boosts', 'sns_held_all', 'sns_sold_some', 'sns_sold_all', 'PNL_loss_5',
       'PNL_loss_10', 'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought',
       'sns_f10_sold', 'sns_no_bought', 'sns_no_sold', 'tt_no_bought',
       'tt_no_sold', 'sns_pnl_loss', 'tt_pnl_loss'],
      dtype='object')

In [82]:
# Объявляем вспомогательную функцию по определению наличия выбросов в признаке
def there_are_outliers(data, feature) -> bool:
    '''
    Функция определяющая методом Тьюки наличие выбросов в признаке
    Args:
        data (DataFrame): датасет
        feature (str): наименование признака
    Returns:
        boolean: возвращает True если выбросы в признаке есть в наличии 
    '''
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return x.min() < lower_bound or x.max() > upper_bound

In [83]:
warnings.simplefilter("ignore", UserWarning)

In [84]:
alpha = 0.05 

scale_columns = [
    'token_age_b', 
    #'transfers', 'transactions', 
    'buys_m5', 'sells_m5',
       'buys_h1', 'sells_h1', 
       'volume_m5', 
       'volume_h1', 
       'price_change_m5',
       'price_change_h1', 
    #    'price_change_h6', 'price_change_h24', 
       'liquidity',
       'market_cap', 
       'boosts', 
       #'telegram_members', 'twitter_days',
       #'twitter_tweets', 'twitter_followers', 'twitter_smart_followers',
       'sns_held_all', 'sns_sold_some', 'sns_sold_all',
       'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought', 'sns_f10_sold', 'sns_no_bought',
       'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
       'tt_pnl_loss', 
       #sns_most_common', 
]
s = []
# Приступаем к маштабированию признаков,
# исходные признаки не трогаем, создаем отдельные,
# т.к. код в данной ячейке может быть исполнен повторно
for feature in scale_columns:
    # С помощью теста Шапиро-Уилка проверим, нормально ли распределён признак 
    result = stats.shapiro(x[feature])
    if result[1] <= alpha:
        print('Признак {} распределен не нормально,'.format(feature))
        # С помощью метода Тьюки проверим, есть ли выбросы
        if there_are_outliers(x, feature):
            print('в признаке {} имеются выбросы, используем RobustScaler().'
                  .format(feature))
            r_scaler = preprocessing.RobustScaler()
            x[str(feature) + '_scaled'] = r_scaler.fit_transform(
                x[feature].values.reshape(x.shape[0],1))
        else:
            print('в признаке {} нет выбросов, используем MinMaxScaler().'
                  .format(feature))
            mm_scaler = preprocessing.MinMaxScaler()
            x[str(feature) + '_scaled'] = mm_scaler.fit_transform(
                x[feature].values.reshape(x.shape[0],1))
    else:
        print('Признак {} распределен нормально,'.format(feature))
        print('используем StandardScaler().')
        s_scaler = preprocessing.StandardScaler()
        x[str(feature) + '_scaled'] = s_scaler.fit_transform(
            x[feature].values.reshape(x.shape[0],1))  
    s.append(str(feature) + '_scaled')



Признак token_age_b распределен не нормально,
в признаке token_age_b нет выбросов, используем MinMaxScaler().
Признак buys_m5 распределен не нормально,
в признаке buys_m5 имеются выбросы, используем RobustScaler().
Признак sells_m5 распределен не нормально,
в признаке sells_m5 имеются выбросы, используем RobustScaler().
Признак buys_h1 распределен не нормально,
в признаке buys_h1 имеются выбросы, используем RobustScaler().
Признак sells_h1 распределен не нормально,
в признаке sells_h1 имеются выбросы, используем RobustScaler().
Признак volume_m5 распределен не нормально,
в признаке volume_m5 имеются выбросы, используем RobustScaler().
Признак volume_h1 распределен не нормально,
в признаке volume_h1 имеются выбросы, используем RobustScaler().
Признак price_change_m5 распределен не нормально,
в признаке price_change_m5 имеются выбросы, используем RobustScaler().
Признак price_change_h1 распределен не нормально,
в признаке price_change_h1 имеются выбросы, используем RobustScaler().
Призна

In [85]:
x = x.drop(scale_columns, axis=1)

In [86]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307 entries, 0 to 472
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price_b                 307 non-null    float64
 1   is_telegram             307 non-null    int64  
 2   is_twitter              307 non-null    int64  
 3   is_website              307 non-null    int64  
 4   PNL_loss_5              307 non-null    float64
 5   PNL_loss_10             307 non-null    float64
 6   token_age_b_scaled      307 non-null    float64
 7   buys_m5_scaled          307 non-null    float64
 8   sells_m5_scaled         307 non-null    float64
 9   buys_h1_scaled          307 non-null    float64
 10  sells_h1_scaled         307 non-null    float64
 11  volume_m5_scaled        307 non-null    float64
 12  volume_h1_scaled        307 non-null    float64
 13  price_change_m5_scaled  307 non-null    float64
 14  price_change_h1_scaled  307 non-null    float64

In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [88]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

In [89]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [90]:
n_features = x_train.shape[1]
model = LogisticRegression(n_features)

optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [91]:
EPOCHS = 5

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f'Среднее значение функции потерь за эпоху {e}: {loss.data}')
    return model

In [92]:
model = train(model, optim, criterion, x_train, y_train)

Среднее значение функции потерь за эпоху 1: 1.057370901107788
Среднее значение функции потерь за эпоху 2: 10.765968322753906
Среднее значение функции потерь за эпоху 3: 8.899994850158691
Среднее значение функции потерь за эпоху 4: 7.012722492218018
Среднее значение функции потерь за эпоху 5: 3.815769672393799


In [93]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(model, x_test, y_test)
print(f'Точность модели: {plain_accuracy}')

Точность модели: 0.6290322542190552


In [94]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# x = df.drop(columns=["profit"])
# y = df["profit"].copy()

# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создание и обучение модели логистической регрессии
model = LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(x_train, y_train)

# Предсказание классов на тестовой выборке
y_pred = model.predict(x_test)

# Вычисление точности классификации
accuracy = accuracy_score(y_test, y_pred)
print("Точность классификации: {:.2f}%".format(accuracy * 100))

Точность классификации: 72.58%


In [95]:
from sklearn.metrics import classification_report  
from sklearn.metrics import accuracy_score  
from sklearn.neighbors import KNeighborsClassifier  

In [96]:
KNN_model = KNeighborsClassifier(n_neighbors=10)

In [97]:
KNN_model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [98]:
KNN_prediction = KNN_model.predict(x_test)

In [99]:
print(accuracy_score(KNN_prediction, y_test))  
print(classification_report(KNN_prediction, y_test))

0.5161290322580645
              precision    recall  f1-score   support

         0.0       0.85      0.54      0.66        54
         1.0       0.11      0.38      0.17         8

    accuracy                           0.52        62
   macro avg       0.48      0.46      0.41        62
weighted avg       0.76      0.52      0.60        62



In [100]:
SVC_model = SVC()
SVC_model.fit(x_train, y_train)   
SVC_prediction = SVC_model.predict(x_test) 


In [102]:
print(accuracy_score(SVC_prediction, y_test))  
print(confusion_matrix(SVC_prediction, y_test, normalize='pred'))  

0.5645161290322581
[[1.         0.96428571]
 [0.         0.03571429]]
